# Deep Crossing

<img src="../../img/embeddingmlp.png" width="500">

- **Embedding 层**。将稀疏的类别型特征（one-hot）转换成稠密的 Embedding 向量
- **Stack 层**。把不同的 Embedding 特征和数值型特征拼接在 起，形成新的包含全部特征的特征向量
- **Multiple Residual Units 层**。通过多层残差网络对特征向量各个维度进行充分的交叉组合，使模型能够抓取到更多的非线性特征和组合特征的信息，进而使深度学习模型在表达能力上较传统机器习模型大为增强
- **Scoring 层**。用于拟合优化目标，对于 CTR 预估问题，Scoring 层往往使用逻辑回归模型

## Pytorch

In [1]:
import pandas as pd
import numpy as np
import json
import torch
from torch import nn
from torch.utils.data.dataset import Dataset
from collections import OrderedDict

In [2]:
learning_rate = 0.00001
batch_size = 64
num_epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
with open('../data/movielens/feature_map.json') as obj:
    feature_map = json.load(obj)
feature_map

{'dataset_id': 'movielens',
 'num_fields': 26,
 'feature_specs': {'movieId': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 935,
   'index': 0},
  'userId': {'source': 'user',
   'type': 'categorical',
   'vocab_size': 22540,
   'index': 1},
  'rating': {'source': 'user',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 2},
  'timestamp': {'source': 'user',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 3},
  'releaseYear': {'source': 'item',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 5},
  'movieGenre1': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 18,
   'index': 6},
  'movieGenre2': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 18,
   'index': 7},
  'movieGenre3': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 15,
   'index': 8},
  'movieRatingCount': {'source': 'item',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 9},
  'movieAvgRating': {'source': 'item',
   'type': 'numerical',
 

In [4]:
class MovielensDataset(Dataset):
    def __init__(self, url):
        self.df = pd.read_csv(url)
    
    def __getitem__(self, idx):
        x, y = self.df.iloc[idx, :-1].values.astype(np.float32), self.df.iloc[idx, -1].astype(np.float32)
        return x, y
    
    def __len__(self):
        return self.df.shape[0]

In [5]:
train_dataset = MovielensDataset('../data/movielens/data_for_train.csv')
test_dataset = MovielensDataset('../data/movielens/data_for_test.csv')

In [6]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [7]:
class DeepCrossing(nn.Module):
    def __init__(self,
                 feature_map,
                 embedding_dim=10,
                 hidden_units=[256, 128, 64]):
        super(DeepCrossing, self).__init__()
        self.feature_map = feature_map
        self.embedding = nn.ModuleDict()
        for feature, feature_spec in feature_map['feature_specs'].items():
            if feature_spec['type'] == 'numerical':
                self.embedding[feature] = nn.Linear(1, embedding_dim, bias=False)
            elif feature_spec['type'] == 'categorical':
                padding_idx = feature_spec.get('padding_idx', None)
                self.embedding[feature] = nn.Embedding(feature_spec['vocab_size'],
                                                       embedding_dim,
                                                       padding_idx=padding_idx)
        input_dim = feature_map['num_fields'] * embedding_dim
        hidden_units = [input_dim] + hidden_units
        residual_layers = []
        for hidden_dim in hidden_units:
            residual_layers.append(ResidualBlock(input_dim,
                                                 hidden_dim))
        residual_layers.append(nn.Linear(input_dim, 1))
        self.residual_layer = nn.Sequential(*residual_layers)
        self.scoring = nn.Sigmoid()

    def forward(self, X):
        feature_emb_list = []
        for feature, feature_spec in self.feature_map['feature_specs'].items():
            if feature_spec['type'] == 'numerical':
                raw_feature = X[:, feature_spec['index']].float().view(-1, 1)
            elif feature_spec['type'] == 'categorical':
                raw_feature = X[:, feature_spec['index']].long()
            embedding_vec = self.embedding[feature](raw_feature)
            feature_emb_list.append(embedding_vec)
        feature_emb = torch.stack(feature_emb_list, dim=1)
        feature_residual = self.residual_layer(feature_emb.flatten(start_dim=1))
        y_pred = self.scoring(feature_residual).squeeze(1)
        return y_pred


class ResidualBlock(nn.Module):
    def __init__(self,
                 input_dim,
                 hidden_dim):
        super(ResidualBlock, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, input_dim)
        self.relu = nn.ReLU()

    def forward(self, X):
        residual = X
        out = self.fc1(X)
        out = self.relu(out)
        out = self.fc2(out)
        out += residual
        out = self.relu(out)
        return out

In [8]:
model = DeepCrossing(feature_map).to(device)
# Loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (X, y) in enumerate(train_loader):
        X = X.to(device)
        y = y.to(device)

        # Forward pass
        output = model(X)
        loss = criterion(output, y)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 300 == 0:
            print("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}".format(
                epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/5], Step [300/1388] Loss: 0.6822
Epoch [1/5], Step [600/1388] Loss: 0.6414
Epoch [1/5], Step [900/1388] Loss: 0.6364
Epoch [1/5], Step [1200/1388] Loss: 0.5439
Epoch [2/5], Step [300/1388] Loss: 0.2867
Epoch [2/5], Step [600/1388] Loss: 0.2195
Epoch [2/5], Step [900/1388] Loss: 0.2013
Epoch [2/5], Step [1200/1388] Loss: 0.0786
Epoch [3/5], Step [300/1388] Loss: 0.0493
Epoch [3/5], Step [600/1388] Loss: 0.0814
Epoch [3/5], Step [900/1388] Loss: 0.0233
Epoch [3/5], Step [1200/1388] Loss: 0.0599
Epoch [4/5], Step [300/1388] Loss: 0.0501
Epoch [4/5], Step [600/1388] Loss: 0.0272
Epoch [4/5], Step [900/1388] Loss: 0.0792
Epoch [4/5], Step [1200/1388] Loss: 0.0169
Epoch [5/5], Step [300/1388] Loss: 0.0325
Epoch [5/5], Step [600/1388] Loss: 0.0136
Epoch [5/5], Step [900/1388] Loss: 0.0084
Epoch [5/5], Step [1200/1388] Loss: 0.0283


In [9]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device).bool()
        output = model(X)
        y_pred = output > 0.5
        total += y.shape[0]
        correct += (y_pred == y).sum().item()

    print('Accuracy of the model on the test images: {:.2f} %'.format(
        100 * correct / total))

Accuracy of the model on the test images: 99.10 %


## Tensorflow

### 读取数据

In [1]:
import tensorflow as tf

In [2]:
# Training samples path, change to your local path
training_samples_file_path = tf.keras.utils.get_file(
    'trainingSamples.csv', '../data/trainingSamples.csv')
# Test samples path, change to your local path
test_samples_file_path = tf.keras.utils.get_file('testSamples.csv',
                                                 '../data/testSamples.csv')


# load sample as tf dataset
def get_dataset(file_path):
    dataset = tf.data.experimental.make_csv_dataset(file_path,
                                                    batch_size=12,
                                                    label_name='label',
                                                    na_value='0',
                                                    num_epochs=1,
                                                    ignore_errors=True)
    return dataset


# split as test dataset and training dataset
train_data = get_dataset(training_samples_file_path)
test_data = get_dataset(test_samples_file_path)

### 特征工程

In [3]:
# genre features vocabulary 电影类型特征
genre_vocab = [
    'Film-Noir', 'Action', 'Adventure', 'Horror', 'Romance', 'War', 'Comedy',
    'Western', 'Documentary', 'Sci-Fi', 'Drama', 'Thriller', 'Crime',
    'Fantasy', 'Animation', 'IMAX', 'Mystery', 'Children', 'Musical'
]

GENRE_FEATURES = {
    'userGenre1': genre_vocab,
    'userGenre2': genre_vocab,
    'userGenre3': genre_vocab,
    'userGenre4': genre_vocab,
    'userGenre5': genre_vocab,
    'movieGenre1': genre_vocab,
    'movieGenre2': genre_vocab,
    'movieGenre3': genre_vocab
}

# all categorical features
categorical_columns = []
for feature, vocab in GENRE_FEATURES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(  # convert the categorical feature to one-hot feature
        key=feature, vocabulary_list=vocab)
    emb_col = tf.feature_column.embedding_column(cat_col, 10) # convert the one-hot to embedding
    categorical_columns.append(emb_col)

# movie id embedding feature
# Use this when your inputs are integers in the range [0, num_buckets), and you want to use the input value itself as the categorical ID.
movie_col = tf.feature_column.categorical_column_with_identity(
    key='movieId', num_buckets=1001)
movie_emb_col = tf.feature_column.embedding_column(movie_col, 10)
categorical_columns.append(movie_emb_col)

# user id embedding feature
user_col = tf.feature_column.categorical_column_with_identity(
    key='userId', num_buckets=30001)
user_emb_col = tf.feature_column.embedding_column(user_col, 10)
categorical_columns.append(user_emb_col)

# all numerical features
numerical_columns = [
    tf.feature_column.numeric_column('releaseYear'),
    tf.feature_column.numeric_column('movieRatingCount'),
    tf.feature_column.numeric_column('movieAvgRating'),
    tf.feature_column.numeric_column('movieRatingStddev'),
    tf.feature_column.numeric_column('userRatingCount'),
    tf.feature_column.numeric_column('userAvgRating'),
    tf.feature_column.numeric_column('userRatingStddev')
]

### 模型训练

In [4]:
# embedding + MLP model architecture
# define input for keras model
inputs = {
    'movieId':
    tf.keras.layers.Input(name='movieId', shape=(), dtype='int32'),
    'userId':
    tf.keras.layers.Input(name='userId', shape=(), dtype='int32'),
    'userGenre1':
    tf.keras.layers.Input(name='userGenre1', shape=(), dtype='string'),
    'userGenre2':
    tf.keras.layers.Input(name='userGenre2', shape=(), dtype='string'),
    'userGenre3':
    tf.keras.layers.Input(name='userGenre3', shape=(), dtype='string'),
    'userGenre4':
    tf.keras.layers.Input(name='userGenre4', shape=(), dtype='string'),
    'userGenre5':
    tf.keras.layers.Input(name='userGenre5', shape=(), dtype='string'),
    'movieGenre1':
    tf.keras.layers.Input(name='movieGenre1', shape=(), dtype='string'),
    'movieGenre2':
    tf.keras.layers.Input(name='movieGenre2', shape=(), dtype='string'),
    'movieGenre3':
    tf.keras.layers.Input(name='movieGenre3', shape=(), dtype='string'),
    'releaseYear':
    tf.keras.layers.Input(name='releaseYear', shape=(), dtype='int32'),
    'movieRatingCount':
    tf.keras.layers.Input(name='movieRatingCount', shape=(), dtype='int32'),
    'movieAvgRating':
    tf.keras.layers.Input(name='movieAvgRating', shape=(), dtype='float32'),
    'movieRatingStddev':
    tf.keras.layers.Input(name='movieRatingStddev', shape=(), dtype='float32'),
    'userRatingCount':
    tf.keras.layers.Input(name='userRatingCount', shape=(), dtype='int32'),
    'userAvgRating':
    tf.keras.layers.Input(name='userAvgRating', shape=(), dtype='float32'),
    'userRatingStddev':
    tf.keras.layers.Input(name='userRatingStddev', shape=(), dtype='float32')
}

outputs = tf.keras.layers.DenseFeatures(numerical_columns +
                                        categorical_columns)(inputs)

outputs = tf.keras.layers.Dense(128, activation='relu')(outputs)
outputs = tf.keras.layers.Dense(128, activation='relu')(outputs)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(outputs)

model = tf.keras.Model(inputs, outputs)

# compile the model, set loss function, optimizer and evaluation metrics
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[
                  'accuracy',
                  tf.keras.metrics.AUC(curve='ROC'),
                  tf.keras.metrics.AUC(curve='PR')
              ])

# train the model
model.fit(train_data, epochs=5)

# evaluate the model
test_loss, test_accuracy, test_roc_auc, test_pr_auc = model.evaluate(
    test_data)
print('\n\nTest Loss {}, Test Accuracy {}, Test ROC AUC {}, Test PR AUC {}'.
      format(test_loss, test_accuracy, test_roc_auc, test_pr_auc))

Epoch 1/5


/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/functional.py:595: UserWarning: Input dict contained keys ['rating', 'timestamp', 'userRatedMovie1', 'userRatedMovie2', 'userRatedMovie3', 'userRatedMovie4', 'userRatedMovie5', 'userAvgReleaseYear', 'userReleaseYearStddev'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


7403/7403 [==============================] - 28s 3ms/step - loss: 4.6167 - accuracy: 0.5541 - auc: 0.5615 - auc_1: 0.6128
Epoch 2/5
7403/7403 [==============================] - 25s 3ms/step - loss: 0.6884 - accuracy: 0.6452 - auc: 0.6854 - auc_1: 0.7160
Epoch 3/5
7403/7403 [==============================] - 26s 4ms/step - loss: 0.5687 - accuracy: 0.7099 - auc: 0.7704 - auc_1: 0.7888
Epoch 4/5
7403/7403 [==============================] - 29s 4ms/step - loss: 0.5259 - accuracy: 0.7386 - auc: 0.8093 - auc_1: 0.8310
Epoch 5/5
1870/1870 [==============================] - 4s 2ms/step - loss: 0.6035 - accuracy: 0.6959 - auc: 0.7536 - auc_1: 0.7817


Test Loss 0.6035098433494568, Test Accuracy 0.6959447264671326, Test ROC AUC 0.75357985496521, Test PR AUC 0.7817345857620239


### 模型预测

In [5]:
# print some predict results
predictions = model.predict(test_data)
for prediction, goodRating in zip(predictions[:12],
                                  list(test_data)[0][1][:12]):
    print('Predicted good rating: {:.2%}'.format(prediction[0]),
          ' | Actual rating label: ',
          ('Good Rating' if bool(goodRating) else 'Bad Rating'))

Predicted good rating: 74.29%  | Actual rating label:  Bad Rating
Predicted good rating: 39.92%  | Actual rating label:  Good Rating
Predicted good rating: 27.14%  | Actual rating label:  Bad Rating
Predicted good rating: 82.99%  | Actual rating label:  Good Rating
Predicted good rating: 33.82%  | Actual rating label:  Good Rating
Predicted good rating: 89.15%  | Actual rating label:  Bad Rating
Predicted good rating: 27.84%  | Actual rating label:  Bad Rating
Predicted good rating: 69.76%  | Actual rating label:  Bad Rating
Predicted good rating: 39.88%  | Actual rating label:  Bad Rating
Predicted good rating: 69.86%  | Actual rating label:  Good Rating
Predicted good rating: 45.90%  | Actual rating label:  Bad Rating
Predicted good rating: 5.60%  | Actual rating label:  Bad Rating
